# Notebook 2: Machine Learning Models
Includes replication code for:
- Table S2
- Table S3
- Table S4
- Table S6
- Table S7

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import roc_curve, confusion_matrix
from sklearn.linear_model import SGDClassifier, SGDRegressor, LogisticRegression
from sklearn.model_selection import cross_validate, KFold, StratifiedKFold, GridSearchCV, cross_val_predict
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.base import clone
from lightgbm import LGBMRegressor, LGBMClassifier
from joblib import dump, load

In [2]:
def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

### Read Data

In [3]:
df = pd.read_csv('data/interim_analysis_datasets/merged_household.csv')
x = df[df.columns[28:]].copy()
xcolumns = x.columns
ids = df['hhid']

### Define Models

In [4]:
class DropMissing(TransformerMixin, BaseEstimator):

    def __init__(self, cols_to_check, colnames, threshold=None, variance_threshold=None):
        self.cols_to_check = cols_to_check
        self.colnames = colnames
        self.threshold = threshold
        self.variance_threshold = variance_threshold
        self.columns_ = None

    def fit(self, X, y=None):
        X = pd.DataFrame(X, columns=self.colnames)
        self.columns_ = X.columns
        missing = X[self.cols_to_check].isna().mean()
        self.missing_frac = missing
        var = X[self.cols_to_check].var()
        self.variance = var
        self.cols_to_drop = missing[(missing > self.threshold) | (var < self.variance_threshold)].index
        self.cols_to_keep = missing[(missing <= self.threshold) & (var >= self.variance_threshold)].index
        return self

    def transform(self, X, y=None):
        X = pd.DataFrame(X, columns=self.colnames)
        return X.drop(self.cols_to_drop.values, axis=1)

    def get_feature_names(self):
        return list(set(self.columns_) - set(self.cols_to_drop))

In [5]:
class Winsorizer(TransformerMixin, BaseEstimator):
    def __init__(self, limits=None):
        self.limits = limits

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        if self.limits is None:
            self.limits = (0.01, 0.99)
        elif isinstance(self.limits, float):
            self.limits = (self.limits, 1 - self.limits)

        columns = [c for c in X.columns if pd.api.types.is_numeric_dtype(X[c])]
        threshold_dict = {}

        for column in columns:
            low, high = X[column].quantile(self.limits)
            threshold_dict[column] = (low, high)

        self.columns_ = columns
        self.threshold_dict_ = threshold_dict

        return self

    def transform(self, X, y=None):
        X = pd.DataFrame(X)
        X_t = X.copy()
        def trim(x, low, high):
            if pd.isna(x):
                return x
            else:
                x = low if x < low else x
                x = high if x > high else x
                return x
        trim_vec = np.vectorize(trim)

        for column, tup in self.threshold_dict_.items():
            X_t[column] = trim_vec(X_t[column], *tup)

        return X_t

    def get_feature_names(self, feature_in=None):
        return self.columns_

In [6]:
# NOTE: To make thing run faster, I've used reduced hyperparameter grids for this replication code (since the
# synthetic data is uncorrelated with the target, hyperparameter searches won't improve performance anyway). 
# The full hyperparameter grids used in the paper are commented out. Running with the full hyperparameter grids
# on the synthetic data takes several hours on a server with 56 CPUs.

#logistic_grid = {'winsorizer__limits':[(0., 1.), (0.005, .995)],
#                 'dropmissing__threshold':[.5, .8, 1],
#                 'dropmissing__variance_threshold':[0, 0.01, 0.1]}

logistic_grid = {'winsorizer__limits':[(0., 1.)],
                 'dropmissing__threshold':[.5],
                 'dropmissing__variance_threshold':[0]}

logistic = Pipeline([('dropmissing', DropMissing(cols_to_check=xcolumns, colnames=xcolumns)),  
                     ('winsorizer', Winsorizer()), 
                     ('imputer', SimpleImputer(strategy='mean')),
                     ('scaler', MinMaxScaler()),
                     ('model', SGDClassifier(loss='log', penalty='l1', alpha=0.000000000000001, n_jobs=1, 
                                             random_state=1))])

#lasso_grid = {'winsorizer__limits':[(0., 1.), (0.005, .995)],
#              'dropmissing__threshold':[.5, .8, 1],
#              'dropmissing__variance_threshold':[0, 0.01, 0.1],
#              'model__alpha':[.00001, .0001, .001, .01, .1, 1, 10, 100]}

lasso_grid = {'winsorizer__limits':[(0., 1.)],
              'dropmissing__threshold':[.5],
              'dropmissing__variance_threshold':[0.1],
              'model__alpha':[1]}

lasso = Pipeline([('dropmissing', DropMissing(cols_to_check=xcolumns, colnames=xcolumns)),  
                  ('winsorizer', Winsorizer()), 
                  ('imputer', SimpleImputer(strategy='mean')),
                  ('scaler', MinMaxScaler()),
                  ('model', SGDClassifier(loss='log', penalty='l1', n_jobs=1, random_state=1))])

#rf_grid = {'winsorizer__limits':[(0., 1.), (0.005, .995)],
#           'dropmissing__threshold':[.5, .8, 1],
#           'dropmissing__variance_threshold':[0, 0.01, 0.1],
#           'model__n_estimators':[50, 100, 200],
#           'model__max_depth':[2, 4, 6, 8]}

rf_grid = {'winsorizer__limits':[(0., 1.)],
           'dropmissing__threshold':[.5],
           'dropmissing__variance_threshold':[0],
           'model__n_estimators':[50],
           'model__max_depth':[2]}

rf = Pipeline([('dropmissing', DropMissing(cols_to_check=xcolumns, colnames=xcolumns)),  
               ('winsorizer', Winsorizer()), 
               ('imputer', SimpleImputer(strategy='mean')),
               ('scaler', MinMaxScaler()),
               ('model', RandomForestClassifier(n_jobs=-1, random_state=1, criterion='entropy'))])

#lgbm_grid = {'winsorizer__limits':[(0., 1.), (0.005, .995)],
#             'dropmissing__threshold':[.5, .8, 1],
#             'dropmissing__variance_threshold':[0, 0.01, 0.1],
#             'model__n_estimators':[50, 100, 200],
#             'model__min_data_in_leaf':[20, 50, 100], 
#             'model__num_leaves':[5, 10],
#             'model__learning_rate':[0.05, 0.075]}

lgbm_grid = {'winsorizer__limits':[(0., 1.)],
             'dropmissing__threshold':[.5],
             'dropmissing__variance_threshold':[0],
             'model__n_estimators':[50],
             'model__min_data_in_leaf':[20], 
             'model__num_leaves':[5],
             'model__learning_rate':[0.05]}

lgbm = Pipeline([('dropmissing', DropMissing(cols_to_check=xcolumns, colnames=xcolumns)), 
                ('winsorizer', Winsorizer()), 
                ('model', LGBMClassifier(n_jobs=1, random_state=1))])
    
regression_models = {
    'logistic':SGDRegressor(loss='squared_loss', penalty='l1', alpha=0.000000000000001, random_state=1),
    'lasso':SGDRegressor(loss='squared_loss', penalty='l1', random_state=1),
    'rf':RandomForestRegressor(n_jobs=-1, random_state=1),
    'lgbm':LGBMRegressor(n_jobs=-1, random_state=1)
}

### Run Models

In [7]:
for outcome, method in [('ultra_poor', 'classification'), ('asset_index', 'regression'), 
                        ('log_expend', 'regression'), ('below_poverty_line', 'classification'), 
                        ('cwr_group', 'regression')]:
    for base_model, grid, label in [(logistic, logistic_grid, 'logistic'), (lasso, lasso_grid, 'lasso'), 
                                     (rf, rf_grid, 'rf'), (lgbm, lgbm_grid, 'lgbm')]:

        # Define y variable
        y = df[outcome]

        # Get file names
        print('Running ' + outcome + ', ' + label + '...')
        base_folder = 'results/simulations/' + outcome + '/' + label + '/'
        if not os.path.isdir(base_folder[:-1]):
            os.mkdir(base_folder[:-1])

        # Use classification if a classification outcome, else use regression
        base_model = clone(base_model)
        if method == 'classification':
            inner_strat = StratifiedKFold(n_splits=3, shuffle=True, random_state=15)
            outer_strat = StratifiedKFold(n_splits=10, shuffle=True, random_state=14) 
            scoring = 'roc_auc'
            scoring_method = 'predict_proba'
        else:
            base_model.set_params(model=regression_models[label])
            inner_strat = KFold(n_splits=3, shuffle=True, random_state=15)
            outer_strat = KFold(n_splits=10, shuffle=True, random_state=14)
            scoring = 'r2'
            scoring_method = 'predict'

        # Create grid search object
        model = GridSearchCV(estimator=base_model, param_grid=grid, cv=inner_strat, scoring=scoring, 
                             verbose=10, refit=True)

        # Get predictions out of sample over 10-fold CV and save to file
        yhat = cross_val_predict(model, x, y, method=scoring_method, n_jobs=-1, cv=outer_strat)
        if method == 'classification':
            yhat = yhat[:, 1]
        predictions_df = pd.DataFrame([ids, y, yhat]).T
        predictions_df.columns = ['hhid', 'y', 'yhat']
        predictions_df.to_csv(base_folder + 'predictions.csv', index=False)
        if method == 'classification':
            print((label + ' Score: %.4f') % roc_auc_score(y, yhat))
        else:
            print((label + ' Score: %.4f') % r2_score(y, yhat))

        # Retrain model on all data to save information on hyperparameters and feature importances
        model.n_jobs = -1
        model.verbose = 0
        model.fit(x, y)
        
        # Save the model itself
        dump(model, base_folder + 'model')
        
        # Save the dataframe of tuning results
        resultsdf = pd.DataFrame(model.cv_results_)
        resultsdf.to_csv(base_folder + 'tuning.csv', index=False)
        
        # Save the dataframe of feature importances
        try:
            imports = model.best_estimator_.named_steps['model'].feature_importances_
        except:
            imports = model.best_estimator_.named_steps['model'].coef_
        cols_kept = model.best_estimator_.named_steps['dropmissing'].cols_to_keep
        imports = pd.DataFrame([cols_kept, imports]).T
        imports.columns = ['feature', 'importance']
        imports.to_csv(base_folder + 'feature_importances.csv')

Running ultra_poor, logistic...
logistic Score: 0.4910
Running ultra_poor, lasso...
lasso Score: 0.4988
Running ultra_poor, rf...
rf Score: 0.5120
Running ultra_poor, lgbm...
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, winsorizer__limits=(0.0, 1.0), score=0.576, total=   1.8s
[CV] dropmissing__threshold=0.5, dropmissing__varian

[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0), score=0.454, total=   2.3s[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is s

[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0), score=0.559, total=   2.3s[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is s

[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0) [LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 

[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, winsorizer__limits=(0.0, 1.0) logistic Score: -0.1655
Running asset_index, lasso...
lasso Score: -0.0072
Running asset_index, rf...

[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0), score=0.503, total=   2.2s
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winso

...

[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0.1, model__alpha=1, winsorizer__limits=(0.0, 1.0), score=-0.001, total=   1.7slasso Score: -0.0026
Running cwr_group, rf...

[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0), score=1.000, total=   2.6s
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0), score=1.000, total=   2.4s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, m

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    8.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    8.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5

rf Score: -0.0241
Running cwr_group, lgbm...
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value:

Fitting 3 folds for each of 1 candidates, totalling 3 fits[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. C

[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0), score=-0.057, total=   2.4s[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is 

### Machine learning from phone data with individual-level matching

In [8]:
df = pd.read_csv('data/interim_analysis_datasets/merged_individual.csv')
x = df[df.columns[28:]].copy()
xcolumns = x.columns
ids = df['hhid']

In [9]:
outcome, method = 'ultra_poor', 'classification'
for base_model, grid, label in [(logistic, logistic_grid, 'logistic'), (lasso, lasso_grid, 'lasso'), 
                                 (rf, rf_grid, 'rf'), (lgbm, lgbm_grid, 'lgbm')]:

    # Define y variable
    y = df[outcome]

    # Get file names
    print('Running ' + outcome + ', ' + label + '...')
    base_folder = 'results/simulations/' + outcome + '/individual/' + label + '/'
    if not os.path.isdir(base_folder[:-1]):
        os.mkdir(base_folder[:-1])

    # Use classification if a classification outcome, else use regression
    base_model = clone(base_model)
    if method == 'classification':
        inner_strat = StratifiedKFold(n_splits=3, shuffle=True, random_state=15)
        outer_strat = StratifiedKFold(n_splits=10, shuffle=True, random_state=14) 
        scoring = 'roc_auc'
        scoring_method = 'predict_proba'
    else:
        base_model.set_params(model=regression_models[label])
        inner_strat = KFold(n_splits=3, shuffle=True, random_state=15)
        outer_strat = KFold(n_splits=10, shuffle=True, random_state=14)
        scoring = 'r2'
        scoring_method = 'predict'

    # Create grid search object
    model = GridSearchCV(estimator=base_model, param_grid=grid, cv=inner_strat, scoring=scoring, 
                         verbose=10, refit=True)

    # Get predictions out of sample over 10-fold CV and save to file
    yhat = cross_val_predict(model, x, y, method=scoring_method, n_jobs=-1, cv=outer_strat)
    if method == 'classification':
        yhat = yhat[:, 1]
    predictions_df = pd.DataFrame([ids, y, yhat]).T
    predictions_df.columns = ['hhid', 'y', 'yhat']
    predictions_df.to_csv(base_folder + 'predictions.csv', index=False)
    if method == 'classification':
        print((label + ' Score: %.4f') % roc_auc_score(y, yhat))
    else:
        print((label + ' Score: %.4f') % r2_score(y, yhat))

    # Retrain model on all data to save information on hyperparameters and feature importances
    model.n_jobs = -1
    model.verbose = 0
    model.fit(x, y)

    # Save the model itself
    dump(model, base_folder + 'model')

    # Save the dataframe of tuning results
    resultsdf = pd.DataFrame(model.cv_results_)
    resultsdf.to_csv(base_folder + 'tuning.csv', index=False)

    # Save the dataframe of feature importances
    try:
        imports = model.best_estimator_.named_steps['model'].feature_importances_
    except:
        imports = model.best_estimator_.named_steps['model'].coef_
    cols_kept = model.best_estimator_.named_steps['dropmissing'].cols_to_keep
    imports = pd.DataFrame([cols_kept, imports]).T
    imports.columns = ['feature', 'importance']
    imports.to_csv(base_folder + 'feature_importances.csv')

Running ultra_poor, logistic...

[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0.1, model__alpha=1, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0.1, model__alpha=1, winsorizer__limits=(0.0, 1.0), score=-0.005, total=   1.7s
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0.1, model__alpha=1, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0.1, model__alpha=1, winsorizer__limits=(0.0, 1.0), score=-0.006, total=   1.7s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0.1, model__alpha=1, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0.1, model__alpha=1, winsorizer__limits=(0.0, 1.0), score=-0.001, total=   1.7s
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0.1, model__alpha=1, winsorizer__limits=(0.0, 1.0) 
[C

[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, winsorizer__limits=(0.0, 1.0) logistic Score: 0.4856
Running ultra_poor, lasso...

[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__max_depth=2, model__n_estimators=50, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__max_depth=2, model__n_estimators=50, winsorizer__limits=(0.0, 1.0), score=-0.009, total=   3.3s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0), score=-0.050, total=   2.8s
[CV] dropmissing__threshold=0.5, dropmissing__va

[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0.1, model__alpha=1, winsorizer__limits=(0.0, 1.0) lasso Score: 0.5038
Running ultra_poor, rf...
rf Score: 0.5388
Running ultra_poor, lgbm...

[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0), score=-0.057, total=   2.4s
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=

Fitting 3 folds for each of 1 candidates, totalling 3 fits[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. C

[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0), score=0.520, total=   2.7s[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    7.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    7.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    7.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    7.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min

[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0) [LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
lgbm Score: 0.5228
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, mi

### Machine learning with asset data

In [10]:
df = pd.read_csv('data/interim_analysis_datasets/merged_household.csv')
x = df[[col for col in df.columns if 'asset' in col and col != 'asset_index']].astype('float')
xcolumns = x.columns
ids = df['hhid']

In [11]:
#logistic_grid = {'winsorizer__limits':[(0., 1.), (0.005, .995)],
#                 'dropmissing__threshold':[.5, .8, 1],
#                 'dropmissing__variance_threshold':[0, 0.01, 0.1]}

logistic_grid = {'winsorizer__limits':[(0., 1.)],
                 'dropmissing__threshold':[.5],
                 'dropmissing__variance_threshold':[0]}

logistic = Pipeline([('dropmissing', DropMissing(cols_to_check=xcolumns, colnames=xcolumns)),  
                     ('winsorizer', Winsorizer()), 
                     ('imputer', SimpleImputer(strategy='mean')),
                     ('scaler', MinMaxScaler()),
                     ('model', SGDClassifier(loss='log', penalty='l1', alpha=0.000000000000001, n_jobs=1, 
                                             random_state=1))])

#lasso_grid = {'winsorizer__limits':[(0., 1.), (0.005, .995)],
#              'dropmissing__threshold':[.5, .8, 1],
#              'dropmissing__variance_threshold':[0, 0.01, 0.1],
#              'model__alpha':[.00001, .0001, .001, .01, .1, 1, 10, 100]}

lasso_grid = {'winsorizer__limits':[(0., 1.)],
              'dropmissing__threshold':[.5],
              'dropmissing__variance_threshold':[0.1],
              'model__alpha':[1]}

lasso = Pipeline([('dropmissing', DropMissing(cols_to_check=xcolumns, colnames=xcolumns)),  
                  ('winsorizer', Winsorizer()), 
                  ('imputer', SimpleImputer(strategy='mean')),
                  ('scaler', MinMaxScaler()),
                  ('model', SGDClassifier(loss='log', penalty='l1', n_jobs=1, random_state=1))])

#rf_grid = {'winsorizer__limits':[(0., 1.), (0.005, .995)],
#           'dropmissing__threshold':[.5, .8, 1],
#           'dropmissing__variance_threshold':[0, 0.01, 0.1],
#           'model__n_estimators':[50, 100, 200],
#           'model__max_depth':[2, 4, 6, 8]}

rf_grid = {'winsorizer__limits':[(0., 1.)],
           'dropmissing__threshold':[.5],
           'dropmissing__variance_threshold':[0],
           'model__n_estimators':[50],
           'model__max_depth':[2]}

rf = Pipeline([('dropmissing', DropMissing(cols_to_check=xcolumns, colnames=xcolumns)),  
               ('winsorizer', Winsorizer()), 
               ('imputer', SimpleImputer(strategy='mean')),
               ('scaler', MinMaxScaler()),
               ('model', RandomForestClassifier(n_jobs=-1, random_state=1, criterion='entropy'))])

#lgbm_grid = {'winsorizer__limits':[(0., 1.), (0.005, .995)],
#             'dropmissing__threshold':[.5, .8, 1],
#             'dropmissing__variance_threshold':[0, 0.01, 0.1],
#             'model__n_estimators':[50, 100, 200],
#             'model__min_data_in_leaf':[20, 50, 100], 
#             'model__num_leaves':[5, 10],
#             'model__learning_rate':[0.05, 0.075]}

lgbm_grid = {'winsorizer__limits':[(0., 1.)],
             'dropmissing__threshold':[.5],
             'dropmissing__variance_threshold':[0],
             'model__n_estimators':[50],
             'model__min_data_in_leaf':[20], 
             'model__num_leaves':[5],
             'model__learning_rate':[0.05]}

lgbm = Pipeline([('dropmissing', DropMissing(cols_to_check=xcolumns, colnames=xcolumns)), 
                ('winsorizer', Winsorizer()), 
                ('model', LGBMClassifier(n_jobs=1, random_state=1))])
    
regression_models = {
    'logistic':SGDRegressor(loss='squared_loss', penalty='l1', alpha=0.000000000000001, random_state=1),
    'lasso':SGDRegressor(loss='squared_loss', penalty='l1', random_state=1),
    'rf':RandomForestRegressor(n_jobs=-1, random_state=1),
    'lgbm':LGBMRegressor(n_jobs=-1, random_state=1)
}

In [12]:
outcome, method = 'ultra_poor', 'classification'
for base_model, grid, label in [(logistic, logistic_grid, 'logistic'), (lasso, lasso_grid, 'lasso'), 
                                 (rf, rf_grid, 'rf'), (lgbm, lgbm_grid, 'lgbm')]:

    # Define y variable
    y = df[outcome].astype('float')

    # Get file names
    print('Running ' + outcome + ', ' + label + '...')
    base_folder = 'results/simulations/' + outcome + '/asset_data/' + label + '/'
    if not os.path.isdir(base_folder[:-1]):
        os.mkdir(base_folder[:-1])

    # Use classification if a classification outcome, else use regression
    base_model = clone(base_model)
    if method == 'classification':
        inner_strat = StratifiedKFold(n_splits=3, shuffle=True, random_state=15)
        outer_strat = StratifiedKFold(n_splits=10, shuffle=True, random_state=14) 
        scoring = 'roc_auc'
        scoring_method = 'predict_proba'
    else:
        base_model.set_params(model=regression_models[label])
        inner_strat = KFold(n_splits=3, shuffle=True, random_state=15)
        outer_strat = KFold(n_splits=10, shuffle=True, random_state=14)
        scoring = 'r2'
        scoring_method = 'predict'

    # Create grid search object
    model = GridSearchCV(estimator=base_model, param_grid=grid, cv=inner_strat, scoring=scoring, 
                         verbose=10, refit=True)

    # Get predictions out of sample over 10-fold CV and save to file
    yhat = cross_val_predict(model, x, y, method=scoring_method, n_jobs=-1, cv=outer_strat)
    if method == 'classification':
        yhat = yhat[:, 1]
    predictions_df = pd.DataFrame([ids, y, yhat]).T
    predictions_df.columns = ['hhid', 'y', 'yhat']
    predictions_df.to_csv(base_folder + 'predictions.csv', index=False)
    if method == 'classification':
        print((label + ' Score: %.4f') % roc_auc_score(y, yhat))
    else:
        print((label + ' Score: %.4f') % r2_score(y, yhat))

    # Retrain model on all data to save information on hyperparameters and feature importances
    model.n_jobs = -1
    model.verbose = 0
    model.fit(x, y)

    # Save the model itself
    dump(model, base_folder + 'model')

    # Save the dataframe of tuning results
    resultsdf = pd.DataFrame(model.cv_results_)
    resultsdf.to_csv(base_folder + 'tuning.csv', index=False)

    # Save the dataframe of feature importances
    try:
        imports = model.best_estimator_.named_steps['model'].feature_importances_
    except:
        imports = model.best_estimator_.named_steps['model'].coef_
    cols_kept = model.best_estimator_.named_steps['dropmissing'].cols_to_keep
    imports = pd.DataFrame([cols_kept, imports]).T
    imports.columns = ['feature', 'importance']
    imports.to_csv(base_folder + 'feature_importances.csv')

Running ultra_poor, logistic...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    4.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    4.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3

logistic Score: 0.5045
Running ultra_poor, lasso...

[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0.1, model__alpha=1, winsorizer__limits=(0.0, 1.0), score=0.500, total=   0.9s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__max_depth=2, model__n_estimators=50, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__max_depth=2, model__n_estimators=50, winsorizer__limits=(0.0, 1.0), score=0.532, total=   1.9s
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__max_depth=2, model__n_estimators=50, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__max_depth=2, model__n_estimators=50, winsorizer__limits=(0.0, 1.0), score=0.502, total=   1.7s
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__max_depth=2, model__n_estimators=50, win

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0), score=0.488, total=   2.5s
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0) 
[CV]  dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=0.05, model__min_data_in_leaf=20, model__n_estimators=50, model__num_leaves=5, winsorizer__limits=(0.0, 1.0), score=0.550, total=   2.3s
[CV] dropmissing__threshold=0.5, dropmissing__variance_threshold=0, model__learning_rate=

Running ultra_poor, rf...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    9.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    9.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    7.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    7.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3

rf Score: 0.4775
Running ultra_poor, lgbm...
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value:

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min

### Table S2

In [13]:
def format_feature(x):
    if x[:19] == 'balance of contacts':
        x = 'BOC' + x[19:]
    elif x[:21] == 'number of interaction':
        x = 'NOI' + x[21:]
    elif x[:24] == 'interactions per contact':
        x = 'IPC' + x[24:]
    elif x[:27] == 'percent pareto interactions':
        x = 'PPI' + x[27:]
    elif x[:24] == 'percent pareto durations':
        x = 'PPD' + x[24:]
    elif x[:15] == 'interevent time':
        x = 'IT' + x[15:]
    elif x[:13] == 'call duration':
        x = 'CD' + x[13:]
    elif x[:14] == 'response delay':
        x = 'RD' + x[14:]
    elif x[:13] == 'response rate':
        x = 'RR' + x[13:]
    tokenized = x.split(' ')
    for i in range(len(tokenized)):
        if tokenized[i] == 'average':
            tokenized[i] = 'avg'
        elif tokenized[i] == 'weekend':
            tokenized[i] = 'WE'
        elif tokenized[i] == 'weekday':
            tokenized[i] = 'WD'
        elif tokenized[i] == 'allweek':
            tokenized[i] = ''
        elif tokenized[i] == 'allday':
            tokenized[i] = ''
        elif tokenized[i] == 'skewness':
            tokenized[i] = 'skew'
        elif tokenized[i] == 'number':
            tokenized[i] = '#'
        elif tokenized[i] == 'percent':
            tokenized[i] = '%'
        elif tokenized[i] == 'of':
            tokenized[i] = ''
        elif tokenized[i] == 'callandtext':
            tokenized[i] = ''
        elif tokenized[i] == ' ':
            tokenized[i] = ''
        if tokenized[i] not in ['BOC', 'NOI', 'IPC', 'PPI', 'PPD', 'IT', 'CD', 'RD', 'RR', 'WE', 'WD']:
            tokenized[i] = tokenized[i].capitalize()
    return ' '.join(tokenized).replace('  ', ' ')

In [14]:
importances = pd.read_csv('results/simulations/ultra_poor/lgbm/feature_importances.csv')\
    [['feature', 'importance']]\
    .dropna(subset=['importance'])\
    .sort_values('importance', ascending=False)
importances['feature'] = importances['feature']\
    .apply(lambda x: ', '.join([format_feature(' '.join([s for s in str(x).split('_') if s != ''])).strip()]))
importances[:50].to_csv('results/tables/tables2.csv', index=False)
importances.head()

,feature,importance
37,CD WE Night Call Median,9
22,CD Night Call Mean,8
16,CD Call Min,8
99,RR Text WD,7
145,BOC WD Night Call Std,7


### Tables S3, S4, S6, and S7

In [15]:
# Set up labels
models = ['logistic', 'lasso', 'rf', 'lgbm']
classification_model_labels = ['Logistic (No Penalty)', 'Logistic (L1 Penalty)', 'Random Forest', 
                               'Gradient Boosting']
regression_model_labels = ['Linear Regression', 'LASSO Regression', 'Random Forest', 'Gradient Boosting']

for outcome, method in [('ultra_poor', 'classification'), ('asset_index', 'regression'), 
                        ('log_expend', 'regression'), ('below_poverty_line', 'regression'), 
                        ('cwr_group', 'regression'), ('ultra_poor/individual', 'classification'),
                        ('ultra_poor/asset_data', 'regression')]:

    # Set up table
    table = []
    for m, model in enumerate(models):

        # Model score
        predictions = pd.read_csv('results/simulations/' + outcome + '/' + model + '/predictions.csv')
        if method == 'classification':
            score = roc_auc_score(predictions['y'], predictions['yhat'])
        else:
            score = r2_score(predictions['y'], predictions['yhat'])
        score = round(score, 2)

        # Top features
        importances = pd.read_csv('results/simulations/ultra_poor/' + model + '/feature_importances.csv')\
            [['feature', 'importance']]\
            .dropna(subset=['importance'])\
            .sort_values('importance', ascending=False)\
            .head(5)
        importances['feature'] = importances['feature']\
            .apply(lambda x: ', '.join([format_feature(' '.join([s for s in str(x).split('_') if s != '']))\
                                        .strip()]))
        importances = ', '.join(list(importances['feature']))

        # Append row
        if method == 'classification':
            table.append([regression_model_labels[m], score, importances])
        else:
            table.append([regression_model_labels[m], score, importances])
            
    # Make table, write to file
    table = pd.DataFrame(table)
    table.columns = ['Model', 'r2 or AUC', 'Top Five Features']
    if outcome == 'ultra_poor':
        table.to_csv('results/tables/tables3.csv', index=False)
    elif outcome == 'ultra_poor/asset_data':
        table.to_csv('results/tables/tables4.csv', index=False)
    elif outcome == 'ultra_poor/individual':
        table.to_csv('results/tables/tables6.csv', index=False)
    else:
        table.to_csv('results/tables/tables7_panel' + outcome + '.csv', index=False)
table

,Model,r2 or AUC,Top Five Features
0,Linear Regression,-0.81,Below Poverty Line
1,LASSO Regression,-0.01,Below Poverty Line
2,Random Forest,-0.01,"BOC WE Night Call Kurtosis, CD Night Call Mean..."
3,Gradient Boosting,-0.01,"CD WE Night Call Median, CD Night Call Mean, C..."


### Combining data sources with machine learning

In [16]:
# Merge survey data and CDR-based predictions
df = pd.read_csv('data/interim_analysis_datasets/merged_household.csv')\
    [['hhid', 'ultra_poor', 'log_expend', 'asset_index', 'weight']]
predictions = pd.read_csv('results/simulations/ultra_poor/lgbm/predictions.csv')[['hhid', 'yhat']]\
    .rename({'yhat':'cdr'}, axis=1)
df = df.merge(predictions, on='hhid', how='inner')

# Set up model, data, and cross validation
y = df['ultra_poor']
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=13)
model = LogisticRegression(penalty='none', n_jobs=1, random_state=100)

# Try different combinations of data together
for regressors in [['asset_index', 'log_expend'], ['asset_index', 'cdr'], ['log_expend', 'cdr'], 
                   ['asset_index', 'log_expend', 'cdr']]:
    x = df[regressors]
    scores = cross_validate(model, x, y, scoring='roc_auc', cv=cv)
    predictions = cross_val_predict(model, x, y, method='predict_proba', cv=cv)
    df['+'.join(regressors)] = predictions[:, 1]
    
# Write to file for later analysis
df.to_csv('data/interim_analysis_datasets/merged_predictions.csv', index=False)